In [8]:
pip install scikit-learn numpy pandas tqdm pyarrow

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.6 MB 2.6 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.6 MB 2.7 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/11.6 MB 2.7 MB/s eta 0:00:04
   ---------- ----------------------------- 3.1/11.6 MB 2.8 MB/s eta 0:00:04
   ------------- -------------------------- 3.9/11.6 MB 2.9 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.6 MB 3.0 MB/s eta 0:00:03
   ------------------- -------------------- 5.8/11.6 MB 3.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.6 MB 3.3 MB/s eta 0:00:02
   --------------------------- ------------ 7.9/11.6 MB 3.6 MB/s eta 0:00:02
   ----------------------------- ---------- 8.7/11.6 MB 3.6 MB/s eta 0:00:01
   ----------

In [10]:
pip install tensorflow

   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/390.2 MB 1.4 MB/s eta 0:04:38
   ---------------------------------------- 1.0/390.2 MB 1.9 MB/s eta 0:03:21
   ---------------------------------------- 1.6/390.2 MB 2.1 MB/s eta 0:03:06
   ---------------------------------------- 2.1/390.2 MB 2.3 MB/s eta 0:02:49
   ---------------------------------------- 2.6/390.2 MB 2.4 MB/s eta 0:02:42
   ---------------------------------------- 3.4/390.2 MB 2.5 MB/s eta 0:02:34
   ---------------------------------------- 3.9/390.2 MB 2.6 MB/s eta 0:02:27
    --------------------------------------- 5.0/390.2 MB 2.8 MB/s eta 0:02:16
    --------------------------------------- 5.8/390.2 MB 3.0 MB/s eta 0:02:10
    --------------------------------------- 6.6/390.2 MB 3.0 MB/s eta 0:02:07
    --------------------------------------- 7.3/390.2 MB 3.1 MB/s eta 0:02:04


  You can safely remove it manually.
  You can safely remove it manually.


In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from math import cos, pi
from tqdm import tqdm
import pandas as pd, time
import pyarrow as pa
from datetime import datetime
import tensorflow as tf, pandas as pd

timestamp = datetime.now().strftime('%Y-%m-%dT%H:%M')
file_path = "../../../data/Combined/data_fg.feather"

In [35]:
import os
from tensorflow.keras.regularizers import l2


In [12]:
def initialize(env_var, default):
    # Initialize with default, possibly overriding by an environment var
    env_value = os.getenv(env_var)
    if isinstance(default, bool):
        value = bool(env_value=="True" or env_value=="1") if env_value else default
    else:
        value = type(default)(env_value) if env_value else default
    return value

In [15]:
INCLUDE_DEM    = initialize('INCLUDE_DEM',    True)
KERAS_TUNER    = initialize('KERAS_TUNER',    False)
BS_OPTIMIZE    = initialize('BS_OPTIMIZE',    False)
CROSSVAL_FOLDS = initialize('CROSSVAL_FOLDS', 0)
PREPROCESS     = initialize('PREPROCESS',     False)
INCLUDE_XY     = initialize('INCLUDE_XY',     True)
MAX_TRAIN_TIME = initialize('MAX_TRAIN_TIME', 3600)
LR             = initialize('LEARNING_RATE',  0.004)
NLAYERS        = initialize('NLAYERS',        5)
MAX_EPOCHS     = initialize('MAX_EPOCHS',     500)
PENALTY        = initialize('PENALTY',        1e-4)  # 5e-4
BATCH_SIZE     = initialize('BATCH_SIZE',     16384)
PARAMS         = initialize('PARAMS',         'all')
PARAMS10       = initialize('PARAMS10',       True)
TAG            = initialize('TAG',            timestamp)
TESTING        = initialize('TESTING',        False)

In [37]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [16]:
def scale(X, scaler, max_col_to_scale):
    X[:, :max_col_to_scale] = scaler.transform(X[:, :max_col_to_scale])
    if max_col_to_scale < X.shape[1]: # Change elevation to km
        X[:, max_col_to_scale:] /= 1000
    return X

In [17]:
def scale_train(X_train, X_val, max_col_to_scale):
    scaler = StandardScaler()
    scaler.fit(X_train[:, :max_col_to_scale])
    X_train = scale(X_train, scaler, max_col_to_scale)
    X_val = scale(X_val, scaler, max_col_to_scale)
    return X_train, X_val, scaler

In [18]:
def splitNscale(X, y, max_col_to_scale):
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2,
                                                      random_state = 42)
    scaler.fit(X_train[:, :max_col_to_scale])
    X_train = scale(X_train, scaler, max_col_to_scale)
    X_val = scale(X_val, scaler, max_col_to_scale)
    X_test = scale(X_test, scaler, max_col_to_scale)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [28]:
def build_model(optimizer, penalty):
    n_units, n_layers, penalty, activation, optimizer = 96, NLAYERS, penalty, 'relu', optimizer
    # n_units, n_layers, penalty, activation, optimizer = 64, 10, 1e-4, 'relu', 'adamax'
    Input = tf.keras.layers.Input
    Dense = tf.keras.layers.Dense
    Dropout = tf.keras.layers.Dropout
    BatchNormalization = tf.keras.layers.BatchNormalization
    model = tf.keras.Sequential()
    
    model.add(Input(shape = (X.shape[1], )))
    model.add(Dense(units=n_units, activation=activation, kernel_regularizer=l2(penalty)))
    model.add(BatchNormalization())
    for _ in range(n_layers):
        model.add(Dense(units=n_units, activation=activation, kernel_regularizer=l2(penalty)))
        model.add(BatchNormalization())
    model.add(Dense(units=n_units, activation=activation, kernel_regularizer=l2(penalty)))
    model.add(Dropout(0.5))
    model.add(Dense(units = 1, activation = 'linear'))
    
    model.compile(optimizer = optimizer, loss = mean_absolute_percentage_error)
    return model

In [49]:
df = pd.read_feather(file_path)
df = df.drop(['stod', 'time', 'X', 'Y', 'from_center'], axis=1)
df = df.dropna()
X, y = df.drop('gust_factor', axis=1).values, df['gust_factor'].values

In [50]:
model = build_model(optimizer = 'adamax', penalty =  0.00012506)

In [51]:
X_train, y_train, X_val, y_val, X_test, y_test = splitNscale(X, y, -1)

In [53]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 200,#MAX_EPOCHS,
                        batch_size = 2**12, verbose = 1)
res = model.evaluate(X_test, y_test, batch_size = 2**12)

Epoch 1/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 20.9059 - val_loss: 14.3882
Epoch 2/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 19.3976 - val_loss: 14.0289
Epoch 3/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 18.3386 - val_loss: 13.7415
Epoch 4/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 17.4895 - val_loss: 13.4610
Epoch 5/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 16.5753 - val_loss: 13.2642
Epoch 6/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 15.7824 - val_loss: 13.1084
Epoch 7/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 15.0636 - val_loss: 13.0284
Epoch 8/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 14.4629 - val_loss: 12.9102
Epoch 9/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 13.9287 - val_loss: 12.8161
Epoch 10/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 13.5424 - val_loss: 12.7179
Epoch 11/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 13.2887 - val_loss: 12.6542
Epoch 12

KeyboardInterrupt: 